In [58]:
import pandas as pd
import numpy as np

In [59]:
df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")

C:\Users\jarne\AppData\Local\Temp\ipykernel_40836\3066643026.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NASA Exoplanet Data/NASA Exoplanet Archive - Planetary Systems Data.csv")


In [60]:
columns_to_select = ['rowid', 'pl_name', 'st_teff','st_lum','sy_dist','pl_orbsmax','pl_orbsmaxerr1','pl_orbsmaxerr2','pl_orbeccen','pl_orbper','pl_masse','pl_rade','pl_dens','pl_eqt']
selected_columns = df[columns_to_select]

print(selected_columns)
print(selected_columns.isnull().sum())
print((selected_columns.isnull().sum() / len(df)) * 100)

       rowid    pl_name  st_teff  st_lum   sy_dist  pl_orbsmax  \
0          1   11 Com b      NaN     NaN   93.1846       1.210   
1          2   11 Com b   4742.0   2.243   93.1846       1.290   
2          3   11 Com b   4874.0   1.978   93.1846       1.178   
3          4   11 UMi b   4213.0     NaN  125.3210       1.530   
4          5   11 UMi b      NaN     NaN  125.3210       1.510   
...      ...        ...      ...     ...       ...         ...   
36457  36458  ups And d      NaN     NaN   13.4054       2.570   
36458  36459  ups Leo b   4836.0   1.800   52.5973       1.180   
36459  36460   xi Aql b      NaN     NaN   56.1858       0.580   
36460  36461   xi Aql b   4780.0   1.839   56.1858       0.680   
36461  36462   xi Aql b   4841.0   1.767   56.1858       0.625   

       pl_orbsmaxerr1  pl_orbsmaxerr2  pl_orbeccen   pl_orbper  pl_masse  \
0                0.06          -0.050          NaN         NaN       NaN   
1                0.05          -0.050        0.231   32

In [61]:
proxima_cen_b_rows = selected_columns[selected_columns['pl_name'] == "Kepler-452 b"]

# Print the rows
print(proxima_cen_b_rows)

       rowid       pl_name  st_teff  st_lum  sy_dist  pl_orbsmax  \
25681  25682  Kepler-452 b   5589.0     NaN  551.727         NaN   
25682  25683  Kepler-452 b   5578.0     NaN  551.727      0.9892   
25683  25684  Kepler-452 b   5579.0     NaN  551.727      0.9940   
25684  25685  Kepler-452 b   5728.0   0.084  551.727         NaN   
25685  25686  Kepler-452 b   5892.0     NaN  551.727         NaN   
25686  25687  Kepler-452 b   5757.0     NaN  551.727      1.0460   

       pl_orbsmaxerr1  pl_orbsmaxerr2  pl_orbeccen   pl_orbper  pl_masse  \
25681             NaN             NaN          NaN         NaN       NaN   
25682             NaN             NaN          0.0  384.843022       NaN   
25683             NaN             NaN          0.0  384.847556       NaN   
25684             NaN             NaN          NaN  384.847556       NaN   
25685             NaN             NaN          NaN  384.843022       NaN   
25686           0.019          -0.015          NaN  384.843000     

In [62]:
INNER_CONSTANT = 1.1
OUTER_CONSTANT = 0.53

def habitable_zone(luminosity):
    luminosity = 10 ** luminosity
    r_inner = np.sqrt(luminosity / INNER_CONSTANT)
    r_outer = np.sqrt(luminosity / OUTER_CONSTANT)
    return r_inner, r_outer

In [63]:
df_grouped = selected_columns.groupby('pl_name').agg({
    'rowid':'min',
    'st_teff': 'mean',
    'st_lum': 'mean',
    'sy_dist':'mean',
    'pl_orbsmax':'mean',
    'pl_orbsmaxerr1':'mean',
    'pl_orbsmaxerr2':'mean',
    'pl_orbeccen':'mean',
    'pl_orbper':'mean',
    'pl_masse':'mean',
    'pl_rade':'mean',
    'pl_dens':'mean',
    'pl_eqt':'mean'
}).reset_index()

In [64]:
df = df_grouped.dropna(subset=['st_lum'])

hz_inner = []
hz_outer = []

for luminosity in df['st_lum']:
    r_inner, r_outer = habitable_zone(luminosity)
    hz_inner.append(r_inner)
    hz_outer.append(r_outer)

df['hz_inner'] = hz_inner
df['hz_outer'] = hz_outer

proxima_cen_b_rows = df[df['pl_name'] == "Kepler-452 b"]

# Print the rows
print(proxima_cen_b_rows)

           pl_name  rowid      st_teff  st_lum  sy_dist  pl_orbsmax  \
4024  Kepler-452 b  25682  5687.166667   0.084  551.727    1.009733   

      pl_orbsmaxerr1  pl_orbsmaxerr2  pl_orbeccen   pl_orbper  pl_masse  \
4024           0.019          -0.015          0.0  384.844831       NaN   

      pl_rade  pl_dens  pl_eqt  hz_inner  hz_outer  
4024   1.3722      NaN   235.0  1.050277  1.513081  


C:\Users\jarne\AppData\Local\Temp\ipykernel_40836\3297939510.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hz_inner'] = hz_inner
C:\Users\jarne\AppData\Local\Temp\ipykernel_40836\3297939510.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hz_outer'] = hz_outer


In [65]:
df['is_habitable'] = (df['pl_orbsmax'] >= df['hz_inner']) & (df['pl_orbsmax'] <= df['hz_outer'])

habitable_planets = df[df['is_habitable']]

print(habitable_planets)
habitable_planets.to_csv("habitable_planets.csv", index=True)

           pl_name  rowid      st_teff    st_lum    sy_dist  pl_orbsmax  \
36        55 Cnc f    124  5248.315000 -0.197000   12.58550    0.778586   
56    BD+14 4559 b    184  4936.000000 -0.320000   49.35200    0.778500   
187      GJ 1002 b    510  3024.000000 -2.852000    4.84867    0.045700   
191      GJ 1061 d    514  2953.000000 -2.770000    3.67278    0.054000   
209       GJ 180 c    571  3371.000000 -1.886000   11.94070    0.129000   
...            ...    ...          ...       ...        ...         ...   
5152    TOI-1736 c  34027  5704.333333  0.260500   88.94820    1.373667   
5195     TOI-199 c  34123  5255.000000 -0.338000  102.27000    0.807000   
5448     TOI-700 d  34709  3470.500000 -1.636000   31.12650    0.163200   
5492  TRAPPIST-1 e  34829  2548.333333 -3.265333        NaN    0.028986   
5718   Wolf 1069 b  36242  3158.000000 -2.531000    9.58341    0.067200   

      pl_orbsmaxerr1  pl_orbsmaxerr2  pl_orbeccen   pl_orbper     pl_masse  \
36          0.010757 

C:\Users\jarne\AppData\Local\Temp\ipykernel_40836\3884127847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_habitable'] = (df['pl_orbsmax'] >= df['hz_inner']) & (df['pl_orbsmax'] <= df['hz_outer'])
